In [46]:
import pandas
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB

Загрузите датасет.

Считайте датасет в Python (можете сразу грузить все в память, выборка небольшая), выясните, что используется в качестве разделителей и как проставляются метки классов.

In [6]:
sms_collection = pandas.read_table(
    'SMSSpamCollection.txt',
    header=None,
    names=['class', 'text'],
)
sms_collection.head()

,class,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте `1` для спама и `0` для "не спама".

Используя `sklearn.feature_extraction.text.CountVectorizer` со стандартными настройками, получите из списка текстов матрицу признаков `X`.

In [31]:
vectorizer = CountVectorizer()
x = vectorizer.fit_transform(sms_collection['text'])
y = (sms_collection['class'] == 'spam').astype(int)

Оцените качество классификации текстов с помощью `LogisticRegression()` с параметрами по умолчанию, используя `sklearn.cross_validation.cross_val_score` и посчитав среднее арифметическое качества на отдельных fold'ах. Установите `random_state=2`. Параметр `cv` задайте равным `10`. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.

In [32]:
answer = cross_val_score(LogisticRegression(random_state=2), x, y, cv=10, scoring='f1').mean()
open('q5.txt', 'wt').write(f'{answer:.1f}');

А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

```
'FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB'
'FreeMsg: Txt: claim your reward of 3 hours talk time'
'Have you visited the last lecture on physics?'
'Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$'
'Only 99$'
```

Прогнозы классификатора (`0` - не спам, `1` - спам), записанные через пробел, будут ответом в одном из вопросов ниже.

In [36]:
estimator = LogisticRegression(random_state=2).fit(x, y)
answer = estimator.predict(vectorizer.transform([
    'FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB',
    'FreeMsg: Txt: claim your reward of 3 hours talk time',
    'Have you visited the last lecture on physics?',
    'Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$',
    'Only 99$',
]))
open('q6.txt', 'wt').write(' '.join(map(str, answer)));

Задайте в `CountVectorizer` параметр `ngram_range=(2, 2)`, затем `ngram_range=(3, 3)`, затем `ngram_range=(1, 3)`. Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки `n`-граммы для разных диапазонов `n` – только биграммы, только триграммы, и, наконец, все вместе – униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [41]:
answer = [
    cross_val_score(
        LogisticRegression(random_state=2),
        CountVectorizer(ngram_range=ngram_range).fit_transform(sms_collection['text']),
        y,
        cv=10,
        scoring='f1',
    ).mean()
    for ngram_range in [(2, 2), (3, 3), (1, 3)]
]
open('q7.txt', 'wt').write(' '.join(f'{score:.2f}' for score in answer));

Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии `MultinomialNB()`. Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

In [43]:
answer = [
    cross_val_score(
        MultinomialNB(),
        CountVectorizer(ngram_range=ngram_range).fit_transform(sms_collection['text']),
        y,
        cv=10,
        scoring='f1',
    ).mean()
    for ngram_range in [(2, 2), (3, 3), (1, 3)]
]
open('q8.txt', 'wt').write(' '.join(f'{score:.2f}' for score in answer));

Попробуйте использовать в логистической регрессии в качестве признаков TF-IDF из `TfidfVectorizer` на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с `CountVectorizer` на униграммах? (напишите в файле с ответом `1`, если повысилось, `-1`, если понизилось, и `0`, если изменилось не более чем на `0.01`). Обратите внимание, что результат перехода к TF-IDF не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [45]:
cross_val_score(
    LogisticRegression(),
    CountVectorizer(ngram_range=(1, 1)).fit_transform(sms_collection['text']),
    y,
    cv=10,
    scoring='f1',
).mean()

0.9326402983610631

In [47]:
cross_val_score(
    LogisticRegression(),
    TfidfVectorizer(ngram_range=(1, 1)).fit_transform(sms_collection['text']),
    y,
    cv=10,
    scoring='f1',
).mean()

0.8528599554172456

In [49]:
open('q9.txt', 'wt').write('-1');